In [1]:
import pandas as pd 
import numpy as np 
import tensorflow as tf
import os
import s3fs
import matplotlib.pyplot as plt 

from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.layers import Dense, GRU, Dropout, Bidirectional, BatchNormalization
from tensorflow.keras.activations import linear, relu, sigmoid
from tensorflow.keras.regularizers import l2

from modules.utils import pre_process_data, encoded_categorical_features

In [2]:
x_train = pd.read_csv("/Users/elouan/Data/Data_Hickathon5/X_train_Hi5.csv", sep=",")

/var/folders/41/lv7xy5r91x7ckkgx5zqmgpxh0000gn/T/ipykernel_98608/3534149467.py:1: DtypeWarning: Columns (108,110,111,115,117,118,122,124,125) have mixed types. Specify dtype option on import or set low_memory=False.
  x_train = pd.read_csv("/Users/elouan/Data/Data_Hickathon5/X_train_Hi5.csv", sep=",")


In [15]:
x_train["piezo_station_update_date"]

0          Sun Jul 14 13:00:02 CEST 2024
1          Sun Jul 14 13:00:02 CEST 2024
2          Sun Jul 14 13:00:02 CEST 2024
3          Sun Jul 14 13:00:02 CEST 2024
4          Sun Jul 14 13:00:02 CEST 2024
                       ...              
2830311    Thu Sep 19 11:45:47 CEST 2024
2830312    Thu Sep 19 14:41:55 CEST 2024
2830313    Thu Sep 19 14:29:16 CEST 2024
2830314     Thu Dec 14 19:02:16 CET 2023
2830315     Tue Jan 28 19:01:33 CET 2020
Name: piezo_station_update_date, Length: 2830316, dtype: object

In [19]:
x_train["piezo_station_update_date"] = x_train["piezo_station_update_date"].str.replace(r"CEST", "", regex=True).str.strip()

# Convert to datetime
x_train["piezo_station_update_date"] = pd.to_datetime(x_train["piezo_station_update_date"], errors="coerce")

In [22]:
x_train = x_train[x_train["piezo_station_update_date"].dt.month.isin([6, 7, 8, 9])]

In [25]:
x_train["piezo_station_update_date"].value_counts()

piezo_station_update_date
2024-06-28 07:31:38    2042184
2024-07-14 13:00:02      69484
2024-07-28 13:00:01      23335
2024-07-17 07:03:49      13166
2024-07-01 19:28:26      12369
                        ...   
2021-06-15 08:06:18        949
2024-07-11 20:04:49        905
2024-07-09 07:03:00        762
2024-08-09 11:31:01        657
2024-08-06 19:29:52        302
Name: count, Length: 63, dtype: int64

In [23]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2247431 entries, 0 to 2830297
Columns: 136 entries, row_index to piezo_groundwater_level_category
dtypes: datetime64[ns](1), float64(90), int64(6), object(39)
memory usage: 2.3+ GB


In [26]:
x_trained_without_nan, y_train = pre_process_data(x_train)

/Users/elouan/anaconda3/envs/Hickathon_env/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/elouan/anaconda3/envs/Hickathon_env/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/elouan/anaconda3/envs/Hickathon_env/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/elouan/anaconda3/envs/Hickathon_env/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/elouan/anaconda3/envs/Hickathon_env/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keep

['prelev_volume_0', 'prelev_usage_label_0', 'prelev_volume_obtention_mode_label_0', 'prelev_volume_1', 'prelev_usage_label_1', 'prelev_volume_obtention_mode_label_1', 'prelev_volume_2', 'prelev_usage_label_2', 'prelev_volume_obtention_mode_label_2', 'prelev_other_volume_sum']


/Users/elouan/anaconda3/envs/Hickathon_env/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/elouan/anaconda3/envs/Hickathon_env/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/elouan/anaconda3/envs/Hickathon_env/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/elouan/anaconda3/envs/Hickathon_env/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/elouan/anaconda3/envs/Hickathon_env/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keep

In [ ]:

y_train.head()

In [ ]:
x_trained_without_nan.head()

In [ ]:
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)     # Show all rows
pd.set_option('display.width', None)        # No line wrap
pd.set_option('display.max_colwidth', None) # No truncation of columns

nan_percentage_5 = x_trained_without_nan.isna().mean() * 100

# Display the percentage of NaN values per column
print(nan_percentage_5.sort_values(ascending=False))

In [30]:

x_trained_without_nan["insee_%_agri"] = x_trained_without_nan["insee_%_agri"].replace(
    {'N/A - division par 0': 0}  # Replace with 0 or any value you choose
).astype(float)
x_trained_without_nan["insee_med_living_level"] = x_trained_without_nan["insee_med_living_level"].replace(
    {'N/A - résultat non disponible': 0}  # Replace with 0 or any value you choose
).astype(float)
x_trained_without_nan["insee_%_ind"] = x_trained_without_nan["insee_%_ind"].replace(
    {'N/A - division par 0': 0}  # Replace with 0 or any value you choose
).astype(float)
x_trained_without_nan["insee_%_const"] = x_trained_without_nan["insee_%_const"].replace(
    {'N/A - division par 0': 0}  # Replace with 0 or any value you choose
).astype(float)

In [31]:
X_final = encoded_categorical_features(x_trained_without_nan)

In [32]:
X_final = pd.DataFrame(X_final)
X_final.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2247431 entries, 0 to 2247430
Data columns (total 86 columns):
 #   Column  Dtype  
---  ------  -----  
 0   0       float64
 1   1       float64
 2   2       float64
 3   3       float64
 4   4       float64
 5   5       float64
 6   6       float64
 7   7       float64
 8   8       float64
 9   9       float64
 10  10      float64
 11  11      float64
 12  12      float64
 13  13      float64
 14  14      float64
 15  15      float64
 16  16      float64
 17  17      float64
 18  18      float64
 19  19      float64
 20  20      float64
 21  21      float64
 22  22      float64
 23  23      float64
 24  24      float64
 25  25      float64
 26  26      float64
 27  27      float64
 28  28      float64
 29  29      float64
 30  30      float64
 31  31      float64
 32  32      float64
 33  33      float64
 34  34      float64
 35  35      float64
 36  36      float64
 37  37      float64
 38  38      float64
 39  39      float64
 40  

In [33]:
X_final = X_final.fillna(X_final.mean())

In [ ]:
import seaborn as sns

# Correlation heatmap
plt.figure(figsize=(15, 10))
sns.heatmap(X_final.corr(), annot=False, cmap='coolwarm', linewidths=0.5)
plt.title("Correlation Matrix")
plt.show()

In [34]:
dataset = tf.data.Dataset.from_tensor_slices((X_final,y_train ))

In [35]:
# Total number of samples in your dataset
dataset_size = len(X_final)
train_size = int(0.6 * dataset_size)  # 60% for training
val_size = int(0.2 * dataset_size)    # 20% for validation
test_size = dataset_size - train_size - val_size  # Remaining 20% for testing

# Shuffle the dataset
dataset = dataset.shuffle(buffer_size=dataset_size)

# Split the dataset into train, validation, and test sets
train_dataset = dataset.take(train_size)    # First 60%
val_dataset = dataset.skip(train_size).take(val_size)  # Next 20%
test_dataset = dataset.skip(train_size + val_size)  # Remaining 20%

# Batch the datasets
batch_size = 32
train_dataset = train_dataset.batch(batch_size)
val_dataset = val_dataset.batch(batch_size)
test_dataset = test_dataset.batch(batch_size)

In [12]:
model = Sequential([
    Dense(128, activation='relu', input_shape=(89,)),  # First hidden layer
    Dense(64, activation='relu'),                      # Second hidden layer
    Dense(5, activation='softmax')                     # Output layer for 5 classes
])

In [ ]:
model.summary()

In [ ]:
def f1_score(y_true, y_pred):
    y_pred = K.argmax(y_pred, axis=-1)
    true_positives = K.sum(K.cast(K.equal(y_true, y_pred), K.floatx()))
    possible_positives = K.sum(K.cast(K.not_equal(y_true, 0), K.floatx()))  # assuming 0 is background
    return true_positives / (K.sum(K.cast(K.equal(y_true, y_pred), K.floatx())) + K.epsilon())

In [13]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
    metrics=['accuracy']
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',  # Choose the metric to monitor
    factor=0.5,          # Factor by which to reduce the learning rate
    patience=3,          # Number of epochs with no improvement before reducing
    min_lr=1e-6         # Minimum learning rate
)

checkpoint = ModelCheckpoint(
    '../models/first_model_3.keras',             # File to save the best model
    monitor='val_accuracy',       # Metric to monitor for improvement
    mode='max',                   # Mode 'max' for accuracy (since higher is better)
    save_best_only=True,          # Save only when there is an improvement
    verbose=1,                    # Print message when saving
                    
)


history = model.fit(
    train_dataset,
    validation_data = val_dataset,
    epochs=5,
    callbacks=[reduce_lr, checkpoint]
)

Epoch 1/5
53069/53069 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3159 - loss: 1.5016
Epoch 1: val_accuracy improved from -inf to 0.37105, saving model to ../models/first_model_3.keras
53069/53069 ━━━━━━━━━━━━━━━━━━━━ 126s 2ms/step - accuracy: 0.3159 - loss: 1.5016 - val_accuracy: 0.3711 - val_loss: 1.4056 - learning_rate: 5.0000e-04
Epoch 2/5
53069/53069 ━━━━━━━━━━━━━━━━━━━━ 0s 764us/step - accuracy: 0.3758 - loss: 1.3945
Epoch 2: val_accuracy improved from 0.37105 to 0.39175, saving model to ../models/first_model_3.keras
53069/53069 ━━━━━━━━━━━━━━━━━━━━ 87s 1ms/step - accuracy: 0.3758 - loss: 1.3945 - val_accuracy: 0.3917 - val_loss: 1.3657 - learning_rate: 5.0000e-04
Epoch 3/5
53069/53069 ━━━━━━━━━━━━━━━━━━━━ 0s 860us/step - accuracy: 0.3908 - loss: 1.3655
Epoch 3: val_accuracy improved from 0.39175 to 0.39811, saving model to ../models/first_model_3.keras
53069/53069 ━━━━━━━━━━━━━━━━━━━━ 76s 1ms/step - accuracy: 0.3908 - loss: 1.3655 - val_accuracy: 0.3981 - val_loss: 1.3506 - l

2024-11-30 22:40:03.879286: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:1: Filling up shuffle buffer (this may take a while): 2572931 of 2830316
2024-11-30 22:40:05.808788: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.



Epoch 4: val_accuracy improved from 0.39811 to 0.40329, saving model to ../models/first_model_3.keras
53069/53069 ━━━━━━━━━━━━━━━━━━━━ 199s 4ms/step - accuracy: 0.3985 - loss: 1.3486 - val_accuracy: 0.4033 - val_loss: 1.3363 - learning_rate: 5.0000e-04
Epoch 5/5


2024-11-30 22:41:07.170769: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:1: Filling up shuffle buffer (this may take a while): 2560120 of 2830316


  138/53069 ━━━━━━━━━━━━━━━━━━━━ 1:00 1ms/step - accuracy: 0.4012 - loss: 1.3422 

2024-11-30 22:41:08.880623: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


53069/53069 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4042 - loss: 1.3361

2024-11-30 22:43:13.441804: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:1: Filling up shuffle buffer (this may take a while): 2650136 of 2830316
2024-11-30 22:43:14.866380: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.



Epoch 5: val_accuracy improved from 0.40329 to 0.40949, saving model to ../models/first_model_3.keras
53069/53069 ━━━━━━━━━━━━━━━━━━━━ 209s 4ms/step - accuracy: 0.4042 - loss: 1.3361 - val_accuracy: 0.4095 - val_loss: 1.3244 - learning_rate: 5.0000e-04


In [ ]:
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Error')
plt.legend()
plt.grid(True)

In [14]:
from sklearn.metrics import f1_score

# 1. Make predictions on the validation dataset
y_pred = model.predict(val_dataset)

# 2. Convert predictions to class labels if necessary
# If the model's last layer is 'softmax', use np.argmax to get class labels
y_pred_classes = np.argmax(y_pred, axis=1)

# 3. Get the true labels from the validation dataset
# Note: This assumes val_dataset is a tf.data.Dataset object containing (features, labels)
y_true = np.concatenate([y for _, y in val_dataset], axis=0)

# 4. Calculate the F1 score
f1 = f1_score(y_true, y_pred_classes, average='weighted')  # 'weighted' is typically used for imbalanced classes

# Print the F1 score
print(f'F1 Score: {f1}')

2024-11-30 22:44:56.823575: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:1: Filling up shuffle buffer (this may take a while): 1797034 of 2830316
2024-11-30 22:45:05.922294: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


17690/17690 ━━━━━━━━━━━━━━━━━━━━ 56s 1ms/step


2024-11-30 22:45:56.882899: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:1: Filling up shuffle buffer (this may take a while): 2642230 of 2830316
2024-11-30 22:45:57.666998: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.
2024-11-30 22:46:30.972616: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


F1 Score: 0.20314432058128504


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, make_scorer
from sklearn.metrics import mean_squared_error, accuracy_score 
from xgboost import XGBClassifier


mon_X_train, mon_X_test, mon_y_train, mon_y_test = train_test_split(X_final,y_train, test_size=0.2, random_state=42)

# Standardiser les données
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(mon_X_train)
X_test_scaled = scaler.transform(mon_X_test)

# Initialiser le modèle XGBoost
model = XGBClassifier(
    objective="multi:softmax",
    num_class=5,
    random_state=42,
    eval_metric="mlogloss",
    colsample_bytree=0.7,
    learning_rate=0.2,
    max_depth=7,
    n_estimators=500,
    reg_alpha=0.5,
    reg_lambda=0.0,
    subsample=1.0
)

# Entraîner le modèle
model.fit(X_train_scaled, mon_y_train)

# Prédire sur le jeu de test
y_pred = model.predict(X_test_scaled)

# Calculer le F1-score pondéré
f1 = f1_score(mon_y_test, y_pred, average='weighted')
print(f"F1-Score pondéré sur le jeu de test : {f1:.2f}")

In [ ]:
model.save_model("xgboost_model_2.json")